# Finalize raw data

In [41]:
path = ('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/')

import os
os.chdir(path)

In [42]:
import pandas as pd

df_raw = pd.read_csv('df_raw.csv')

In [43]:
# sexual orientation 6 and 5 are same category across years
df_raw['sexual_orientation_M'].replace({6:5}, inplace=True)
df_raw['sexual_orientation_F'].replace({6:5}, inplace=True)

# fill male NA with female answers
df_raw['sexual_orientation_M'] = df_raw.apply(
    lambda row: row['sexual_orientation_F'] if np.isnan(row['sexual_orientation_M']) else row['sexual_orientation_M'],
    axis=1)

# lifetime partners
# fill male NA with female answers
df_raw['lifetime_male_partners'] = df_raw.apply(
    lambda row: row['lifetime_female_partners'] if np.isnan(row['lifetime_male_partners']) else row['lifetime_male_partners'],
    axis=1)


# fill in doc_diabtetes with pre-diabetes if they are missing
df_raw['doc_diabetes'] = df_raw.apply(
    lambda row: row['doc_prediabetes'] if np.isnan(row['doc_diabetes']) else row['doc_diabetes'],
    axis=1)


# total cholesterol is better than these two, drop female ortientation lifetime partners, and prediabetes
df_raw.drop(['HDL','LDL', 'sexual_orientation_F', 'lifetime_female_partners', 'doc_prediabetes'], axis=1, inplace=True)

df_raw.rename(columns={"sexual_orientation_M": "sexual_orientation", "lifetime_male_partners": "lifetime_partners"}, inplace=True)


## Replace all 7='refused' and 9='don't know' with NaN

In [80]:
import numpy as np
# columns that are not continuous:
non_cont = ['race_ethnicity', 'edu_level','gender', 'marital_status', 'how_healthy_diet','doc_diabetes','difficult_hearing',
           'difficult_seeing', 'used_CMH', 'health_insurance', 'doc_asthma', 'doc_overweight',
            'doc_arthritis', 'doc_CHF', 'doc_CHD', 'doc_heart_attack', 'doc_stroke', 'doc_chronic_bronchitis', 
            'doc_liver_condition', 'doc_thyroid_problem', 'doc_COPD', 'doc_cancer', 'doc_kidney', 
            'broken_hip', 'doc_osteoporosis', 'vigorous_activity', 'moderate_activity','doc_sleeping_disorder', 
            'smoker', 'sexual_orientation', 'lifetime_partners', 'doc_HPV', 'alcoholic', 'herpes_2', 'HIV']


for col in non_cont:
    df_raw[col].replace([7,77,777,7777,77777], [np.nan,np.nan,np.nan,np.nan,np.nan], inplace=True)
    df_raw[col].replace([9,99,999,9999,99999], [np.nan,np.nan,np.nan,np.nan,np.nan], inplace=True)
    
    
# annual_HI - 99 77, used_marijuana - 77 or 777, min_sedetary - 7777
col_2 = ['annual_HI', 'used_marijuana', 'min_sedetary']

for col in col_2:
    df_raw[col].replace([77,777,7777,77777], [np.nan,np.nan,np.nan,np.nan], inplace=True)
    df_raw[col].replace([99,999,9999,99999], [np.nan,np.nan,np.nan,np.nan], inplace=True)

## Replace all 1=yes, 2=no with 1=True, 0=False

In [81]:
# all T/F columns
col_TF = ['doc_diabetes', 'difficult_hearing', 'difficult_seeing','used_CMH', 
          'health_insurance', 'doc_asthma', 'doc_overweight', 'doc_arthritis', 'doc_CHF', 'doc_CHD', 
          'doc_heart_attack', 'doc_stroke', 'doc_chronic_bronchitis', 'doc_liver_condition', 'doc_thyroid_problem', 
          'doc_COPD', 'doc_cancer', 'doc_kidney', 'broken_hip', 'doc_osteoporosis', 'vigorous_activity', 
          'moderate_activity','doc_sleeping_disorder', 'smoker', 'doc_HPV', 'alcoholic', 'herpes_2', 'HIV']

# replace 2 with 0
for col in col_TF:
    # should I replace 'indeterminant'/'boarderline' with 0 or NaN?
    df_raw[col].replace([2,3], [0,0], inplace=True)


In [82]:
df_raw['how_healthy_diet'].value_counts()

3.0    12900
4.0     7477
2.0     6416
1.0     2689
5.0     1857
Name: how_healthy_diet, dtype: int64

In [34]:
df_raw.columns

Index(['SEQN', 'race_ethnicity', 'edu_level', '#_ppl_household', 'age',
       'gender', 'marital_status', 'annual_HI', 'depressed', 'caffeine',
       'doc_diabetes', 'how_healthy_diet', 'used_marijuana', 'used_CMH',
       'health_insurance', 'doc_asthma', 'doc_overweight', 'doc_arthritis',
       'doc_CHF', 'doc_CHD', 'doc_heart_attack', 'doc_stroke',
       'doc_chronic_bronchitis', 'doc_liver_condition', 'doc_thyroid_problem',
       'doc_cancer', 'difficult_seeing', 'doc_kidney', 'broken_hip',
       'doc_osteoporosis', 'vigorous_activity', 'moderate_activity',
       'doc_sleeping_disorder', 'smoker', 'sexual_orientation',
       'lifetime_partners', 'alcoholic', 'triglyceride', 'tot_cholesterol',
       'glycohemoglobin', 'herpes_2', 'HIV', 'CRP', 'systolic_BP',
       'diastolic_BP', 'BMI', 'waist_C', 'year', '#meals_fast_food',
       'min_sedetary', 'doc_HPV', 'bone_mineral_density', 'difficult_hearing',
       'doc_COPD'],
      dtype='object')

In [77]:
import numpy as np

# outliers in lifetime partners
df_raw['lifetime_partners'] = np.where(df_raw['lifetime_partners']>100, np.nan, df_raw['lifetime_partners'])

# 'annual_HI' - 1-10 $5,000 increments, 11-$75,000 (only in 2004) 12-Over $20,000, 
# 13-Under $20,000, 14-$75,000 to $99,999, 15-$100,000 and Over
# drop 12 and 13, make 14-->12, 15-->13
df_raw['annual_HI'] = np.where(df_raw['annual_HI']==12, np.nan, df_raw['annual_HI'])
df_raw['annual_HI'] = np.where(df_raw['annual_HI']==13, np.nan, df_raw['annual_HI'])
df_raw['annual_HI'].replace({14:12, 15:13}, inplace=True)

# outliers in caffeine
df_raw['caffeine'] = np.where(df_raw['caffeine']>500, np.nan, df_raw['caffeine'])

#df_raw['triglyceride'] = np.where(df_raw['triglyceride']>400, np.nan, df_raw['triglyceride'])

df_raw['#meals_fast_food'] = np.where(df_raw['#meals_fast_food']<1,0, df_raw['#meals_fast_food'])
df_raw['#meals_fast_food'] = np.where(df_raw['#meals_fast_food']==9999,np.nan, df_raw['#meals_fast_food'])
df_raw['#meals_fast_food'] = np.where(df_raw['#meals_fast_food']==5555,np.nan, df_raw['#meals_fast_food'])

In [74]:
df_raw.to_csv('/Users/carolinesklaver/Desktop/Capstone/NHANES/data/csv_data/df_raw_v2.csv', index=False)